## Explore K-Means clustering using R and Tidy data principles.

### [**Pre-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

In this lesson, you will learn how to create clusters using the Tidymodels package and other packages in the R ecosystem (we'll call them friends 🧑‍🤝‍🧑), along with the Nigerian music dataset you imported earlier. We will cover the basics of K-Means for clustering. Remember, as you learned in the previous lesson, there are many ways to work with clusters, and the method you choose depends on your data. We'll try K-Means since it's the most common clustering technique. Let's dive in!

Terms you will learn about:

- Silhouette scoring  
- Elbow method  
- Inertia  
- Variance  

### **Introduction**

[K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) is a method derived from the field of signal processing. It is used to divide and group data into `k clusters` based on similarities in their features.

The clusters can be visualized as [Voronoi diagrams](https://wikipedia.org/wiki/Voronoi_diagram), which consist of a point (or 'seed') and its corresponding region.

<p>
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infographic by Jen Looper</figcaption>

K-Means clustering follows these steps:

1. The data scientist begins by specifying the desired number of clusters to be created.  
2. The algorithm then randomly selects K observations from the dataset to serve as the initial centers for the clusters (i.e., centroids).  
3. Each of the remaining observations is assigned to its closest centroid.  
4. The new means of each cluster are calculated, and the centroid is moved to the mean.  
5. Once the centers are recalculated, every observation is checked again to see if it might be closer to a different cluster. All objects are reassigned using the updated cluster means. The cluster assignment and centroid update steps are repeated iteratively until the cluster assignments stop changing (i.e., when convergence is achieved). Typically, the algorithm stops when each new iteration results in negligible movement of centroids, and the clusters stabilize.

<div>

> Note that due to the randomization of the initial k observations used as starting centroids, slightly different results can occur each time the procedure is applied. For this reason, most algorithms use several *random starts* and select the iteration with the lowest WCSS. Therefore, it is strongly recommended to always run K-Means with several values of *nstart* to avoid an *undesirable local optimum.*

</div>

This short animation using the [artwork](https://github.com/allisonhorst/stats-illustrations) of Allison Horst illustrates the clustering process:

<p>
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Artwork by @allison_horst</figcaption>

A key question that arises in clustering is: how do you determine the number of clusters to divide your data into? One limitation of K-Means is that you need to define `k`, the number of `centroids`. Fortunately, the `elbow method` helps estimate a good starting value for `k`. You'll try it shortly.

### **Prerequisite**

We'll pick up right where we left off in the [previous lesson](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), where we analyzed the dataset, created various visualizations, and filtered the dataset to focus on observations of interest. Be sure to check it out!

We'll need some packages to complete this module. You can install them using:  
`install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternatively, the script below checks whether you have the required packages for this module and installs any missing ones for you.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


## 1. A dance with data: Narrow down to the 3 most popular music genres

This is a summary of what we covered in the previous lesson. Let's analyze and break down some data!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 That went well!

## 2. More data exploration.

How clean is this data? Let's check for outliers using box plots. We will focus on numeric columns with fewer outliers (although you could remove the outliers). Boxplots can show the range of the data and will help decide which columns to use. Keep in mind, boxplots do not display variance, which is a key factor for good clusterable data. For more information, check out [this discussion](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Boxplots](https://en.wikipedia.org/wiki/Box_plot) are used to visually represent the distribution of `numeric` data, so let's begin by *selecting* all numeric columns along with the popular music genres.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


See how the selection helper `where` simplifies this process 💁? Discover more functions like this [here](https://tidyselect.r-lib.org/).

Since we’ll be creating a boxplot for each numeric feature and want to avoid using loops, let’s reshape our data into a *longer* format. This will enable us to use `facets`—subplots that showcase individual subsets of the data.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Much longer! Now it's time for some `ggplots`! So, which `geom` are we going to use?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Now we can see that this data is somewhat messy: by looking at each column as a boxplot, you can spot outliers. You could go through the dataset and eliminate these outliers, but that would leave the data quite sparse.

For now, let's decide which columns we'll use for our clustering exercise. We'll select the numeric columns with similar ranges. While we could encode `artist_top_genre` as numeric, we'll leave it out for now.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Computing k-means clustering in R

We can compute k-means in R using the built-in `kmeans` function; refer to `help("kmeans()")`. The `kmeans()` function takes a data frame with all numeric columns as its main argument.

The first step in using k-means clustering is to define the number of clusters (k) that will be created in the final solution. Since we know there are 3 song genres identified in the dataset, let's try using 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


The kmeans object contains several pieces of information, which are well explained in `help("kmeans()")`. For now, let's focus on a few. We can see that the data has been divided into 3 clusters with sizes of 65, 110, and 111. The output also includes the cluster centers (means) for the 3 groups across the 5 variables.

The clustering vector represents the cluster assignment for each observation. Let's use the `augment` function to add the cluster assignment to the original dataset.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfect, we have just divided our dataset into 3 groups. So, how good is our clustering 🤷? Let's take a look at the `Silhouette score`.

### **Silhouette score**

[Silhouette analysis](https://en.wikipedia.org/wiki/Silhouette_(clustering)) can be used to evaluate the separation distance between the resulting clusters. This score ranges from -1 to 1, where a score close to 1 indicates that the cluster is compact and well-separated from other clusters. A score near 0 suggests overlapping clusters, with samples positioned close to the decision boundary of neighboring clusters. [source](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

The average silhouette method calculates the mean silhouette score of observations for different values of *k*. A high average silhouette score signifies effective clustering.

The `silhouette` function in the cluster package computes the average silhouette width.

> The silhouette can be calculated using any [distance](https://en.wikipedia.org/wiki/Distance "Distance") metric, such as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") or [Manhattan distance](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), which we discussed in the [previous lesson](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Our score is **.549**, which places us right in the middle. This suggests that our data isn't particularly well-suited for this type of clustering. Let's check if we can confirm this assumption visually. The [factoextra package](https://rpkgs.datanovia.com/factoextra/index.html) offers functions (`fviz_cluster()`) to visualize clustering.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


The overlap in clusters indicates that our data is not particularly well-suited for this type of clustering, but let's proceed.

## 4. Determining optimal clusters

A common question that arises in K-Means clustering is this: without having predefined class labels, how can you determine the number of clusters to divide your data into?

One approach to address this is to use a data sample to `create a series of clustering models` with an increasing number of clusters (e.g., from 1 to 10) and evaluate clustering metrics such as the **Silhouette score.**

We can determine the optimal number of clusters by running the clustering algorithm for different values of *k* and assessing the **Within Cluster Sum of Squares** (WCSS). The total within-cluster sum of squares (WCSS) measures how compact the clusters are, and we aim for it to be as small as possible. Lower values indicate that the data points within a cluster are closer to each other.

Let's examine how different choices of `k`, ranging from 1 to 10, impact this clustering.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Now that we have the total within-cluster sum-of-squares (tot.withinss) for each clustering algorithm with center *k*, we use the [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) to determine the optimal number of clusters. This method involves plotting the WCSS as a function of the number of clusters and selecting the [elbow of the curve](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") as the ideal number of clusters.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


The plot shows a significant decrease in WCSS (indicating greater *compactness*) as the number of clusters increases from one to two, followed by another noticeable drop from two to three clusters. Beyond that, the reduction becomes less significant, creating an `elbow` 💪 in the chart around three clusters. This suggests that there are likely two to three well-separated clusters of data points.

We can now proceed to extract the clustering model with `k = 3`:

> `pull()`: used to extract a single column
>
> `pluck()`: used to access elements in data structures like lists


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Great! Let's proceed to visualize the clusters we've obtained. How about adding some interactivity with `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Perhaps we might have anticipated that each cluster (indicated by different colors) would correspond to unique genres (indicated by different shapes).

Let's examine the model's accuracy.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


This model's accuracy is decent, but not exceptional. It’s possible that the data isn’t well-suited for K-Means Clustering. The dataset is too imbalanced, the correlations are weak, and there’s too much variance between column values to form effective clusters. In fact, the clusters that do form are likely heavily influenced or skewed by the three genre categories we defined earlier.

That said, this has been a valuable learning experience!

In Scikit-learn's documentation, you’ll find that a model like this one, where clusters are not clearly defined, faces a 'variance' issue:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infographic from Scikit-learn</figcaption>



## **Variance**

Variance is described as "the average of the squared differences from the Mean" [source](https://www.mathsisfun.com/data/standard-deviation.html). In the context of this clustering problem, it means that the values in our dataset tend to deviate too much from the mean.

✅ This is a great opportunity to brainstorm ways to address this issue. Should you adjust the data further? Use different columns? Try a different algorithm? Hint: Consider [scaling your data](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) to normalize it and experiment with other columns.

> Use this '[variance calculator](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' to deepen your understanding of the concept.

------------------------------------------------------------------------

## **🚀Challenge**

Spend some time experimenting with this notebook and tweaking the parameters. Can you improve the model’s accuracy by cleaning the data further (e.g., removing outliers)? You could assign weights to emphasize certain data samples. What other strategies can you think of to create better clusters?

Hint: Try scaling your data. There’s commented code in the notebook that applies standard scaling to make the data columns more comparable in range. You’ll notice that while the silhouette score decreases, the 'kink' in the elbow graph becomes smoother. This happens because unscaled data allows features with less variance to disproportionately influence the clustering. You can read more about this issue [here](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Post-lecture quiz**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Review & Self Study**

-   Explore a K-Means Simulator [like this one](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). This tool allows you to visualize sample data points and determine their centroids. You can adjust the data’s randomness, the number of clusters, and the number of centroids. Does this help you better understand how data can be grouped?

-   Check out [this handout on K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) from Stanford.

Want to test your new clustering skills on datasets that are well-suited for K-Means clustering? Take a look at:

-   [Train and Evaluate Clustering Models](https://rpubs.com/eR_ic/clustering) using Tidymodels and related tools

-   [K-means Cluster Analysis](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering with tidy data principles](https://www.tidymodels.org/learn/statistics/k-means/)

## **Assignment**

[Experiment with different clustering methods](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## THANK YOU TO:

[Jen Looper](https://www.twitter.com/jenlooper) for creating the original Python version of this module ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) for designing the wonderful illustrations that make R more approachable and engaging. You can find more of her work in her [gallery](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Happy Learning,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Artwork by @allison_horst</figcaption>



---

**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we aim for accuracy, please note that automated translations may include errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is advised. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
